# Hecho servicios acumulados

In [1]:
from datetime import date
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import holidays
import yaml

### Conexión con la base de datos

In [2]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [93]:
table_servicio = pd.read_sql_table('mensajeria_servicio', oltp_conn)
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn)
hecho_servicios = table_servicio[['id', 'cliente_id', 'mensajero_id', 'ciudad_destino_id', 'ciudad_origen_id', 'mensajero2_id', 'mensajero3_id']]

estados = [
  (1, 'id_fecha_iniciado'),
  (2, 'id_fecha_mensajero'),
  (3, 'id_fecha_novedad'),
  (4, 'id_fecha_recogida'),
  (5, 'id_fecha_entrega'),
  (6, 'id_fecha_terminado')]

for estado_id, nombre in estados: # genera todas las llaves con la dimensión fecha 
  filtered_estados = (
    table_estadosservicio[['id', 'estado_id', 'servicio_id']]
    .query(f"estado_id == {estado_id}")
    .rename(columns={'id': nombre})
    .drop(columns=['estado_id'])
    .drop_duplicates(subset=['servicio_id']))
  hecho_servicios = (hecho_servicios
    .merge(filtered_estados, how='left', left_on='id', right_on='servicio_id')
    .drop(columns=['servicio_id']))

hecho_servicios.head()
#print(hecho_servicios.shape[0])


,id,cliente_id,mensajero_id,ciudad_destino_id,ciudad_origen_id,mensajero2_id,mensajero3_id,id_fecha_iniciado,id_fecha_mensajero,id_fecha_novedad,id_fecha_recogida,id_fecha_entrega,id_fecha_terminado
0,34,5,NaN,1,1,NaN,NaN,123.0,NaN,NaN,NaN,NaN,NaN
1,35,5,7.0,1,1,NaN,NaN,124.0,125.0,NaN,126.0,195.0,NaN
2,36,5,NaN,1,1,NaN,NaN,127.0,NaN,NaN,NaN,NaN,NaN
3,41,5,NaN,1,1,NaN,NaN,138.0,NaN,NaN,NaN,NaN,NaN
4,42,5,NaN,1,1,NaN,NaN,139.0,NaN,NaN,NaN,NaN,NaN


### Módulo de transformación

### Módulo de carga a la bodega de datos